# Feature engineering for transactions data

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt


In [2]:
path = "data/"

In [3]:
transactions_train = pd.read_csv(path + "transactions_train.csv")
transactions_test = pd.read_csv(path + "transactions_test.csv")

In [4]:
def downsample(transactions, verbose=False):

    # Downsamples our data so it's not as memory intensive as it could be.
    # Verbose=True prints the results of downsampling

    if verbose:
        print('before downsampling:\n', transactions.dtypes)
    transactions['client_id'] = pd.to_numeric(transactions['client_id'], downcast='unsigned')
    transactions['trans_date'] = pd.to_numeric(transactions['trans_date'], downcast='unsigned')
    transactions['small_group'] = pd.to_numeric(transactions['small_group'], downcast='signed')
    transactions['amount_rur'] = pd.to_numeric(transactions['amount_rur'], downcast='float')
    if verbose:
        print('after downsampling:\n',transactions.dtypes)

In [5]:
def filter_groups(transactions, threshold_value=0.01, based_on_volume=True):
    
    # Filters some of the rare vendor groups
    # If based_on_volume is true, filters based on transaction volume of the vendor,
    # otherwise filters based on the number of transactions of the vendor.

    if based_on_volume:
        normalized_sums = transactions.groupby('small_group').sum()['amount_rur']
        normalized_sums /= normalized_sums.sum()
        normalized_sorted_sums = normalized_sums.sort_values()

        threshold = normalized_sorted_sums[(normalized_sorted_sums.cumsum() > threshold_value).idxmax()]

        transactions['small_group_filtered'] = transactions['small_group'].mask(transactions['small_group'].map(normalized_sums) < threshold, -1)
    else:
        frequencies = transactions['small_group'].value_counts(normalize=True, ascending=True)

        threshold = frequencies[(frequencies.cumsum() > threshold_value).idxmax()]
        transactions['small_group_filtered'] = transactions['small_group'].mask(transactions['small_group'].map(transactions['small_group'].value_counts(normalize=True)) < threshold, -1)
    
    transactions['small_group'] = transactions['small_group_filtered']
    transactions = transactions.drop(columns=['small_group_filtered'])

In [6]:
def flag_large_transactions(transactions, quantile):

    # Adds a "large_transaction" feature with flags on transactions that have a larger volume than the specified quantile.
    # For reference, quantile of 0.996 sets flags on transactions larger than 1000 RUB

    transactions['large_amount'] = False
    transactions['large_amount'] = transactions['large_amount'].mask(transactions['amount_rur'] > transactions['amount_rur'].quantile(quantile), True)

In [7]:
def group_mean(transactions):

    # Simply adds the mean of the volume of transactions of a corresponding merchant id to every transaction.
    # Could help us get more useful data about the vendor.
    
    transactions['group_mean'] = transactions['small_group'].map(transactions.groupby('small_group').mean()['amount_rur'])

In [8]:
def process_datetime(transactions, leak_client_id=False):

    # Adds various information about the date of the transaction (in the absence of time)
    # Also includes various statistics based on the date. 
    # leak_client_id set to true will add even more statistics
    # but will leak the value of client_id into new features

    # TODO: add functions for mappings, reducing groups of three lines into one

    # converting the date to the datetime type
    transactions['trans_date'] = pd.to_datetime(transactions['trans_date'], unit='d')

    # processing day of week, finding averages of transactions

    transactions['day_of_week'] = transactions['trans_date'].dt.dayofweek

    baseline = transactions['day_of_week']
    mapping = transactions.groupby('day_of_week').mean()['amount_rur']
    transactions['general_day_of_week_transaction_average'] = baseline.map(mapping)

    baseline = pd.Series(list(zip(transactions['day_of_week'], transactions['small_group'])))
    mapping = transactions.groupby(['day_of_week', 'small_group']).mean()['amount_rur']
    transactions['general_day_of_week_transaction_group_average'] = baseline.map(mapping)

    if leak_client_id:
        baseline = pd.Series(list(zip(transactions['client_id'], transactions['day_of_week'])))
        mapping = transactions.groupby(['client_id','day_of_week']).mean()['amount_rur']
        transactions['client_day_of_week_transaction_average'] = baseline.map(mapping)

        baseline = pd.Series(list(zip(transactions['client_id'], transactions['day_of_week'], transactions['small_group'])))
        mapping = transactions.groupby(['client_id','day_of_week', 'small_group']).mean()['amount_rur']
        transactions['client_day_of_week_transaction_group_average'] = baseline.map(mapping)

    # and daily averages

    day_of_week_counts = pd.Series(transactions['trans_date'].unique()).dt.dayofweek.value_counts()
    unique_clients = transactions['client_id'].nunique()

    baseline = transactions['day_of_week']
    mapping = transactions.groupby('day_of_week').sum()['amount_rur']
    mapping /= (day_of_week_counts * unique_clients)
    transactions['general_day_of_week_daily_average'] = baseline.map(mapping)

    baseline = pd.Series(list(zip(transactions['day_of_week'], transactions['small_group'])))
    mapping = transactions.groupby(['day_of_week', 'small_group']).sum()['amount_rur']
    mapping = mapping.reset_index()
    mapping['amount_rur'] /= (mapping['day_of_week'].map(day_of_week_counts) * unique_clients)
    mapping = mapping.set_index(['day_of_week', 'small_group'])
    mapping = mapping['amount_rur']
    transactions['general_day_of_week_daily_group_average'] = baseline.map(mapping)

    if leak_client_id:
        baseline = pd.Series(list(zip(transactions['client_id'], transactions['day_of_week'])))
        mapping = transactions.groupby(['client_id', 'day_of_week']).sum()['amount_rur']
        mapping = mapping.reset_index()
        mapping['amount_rur'] /= (mapping['day_of_week'].map(day_of_week_counts))
        mapping = mapping.set_index(['client_id', 'day_of_week'])
        mapping = mapping['amount_rur']
        transactions['client_day_of_week_daily_average'] = baseline.map(mapping)

        baseline = pd.Series(list(zip(transactions['client_id'], transactions['day_of_week'], transactions['small_group'])))
        mapping = transactions.groupby(['client_id', 'day_of_week', 'small_group']).sum()['amount_rur']
        mapping = mapping.reset_index()
        mapping['amount_rur'] /= (mapping['day_of_week'].map(day_of_week_counts))
        mapping = mapping.set_index(['client_id', 'day_of_week', 'small_group'])
        mapping = mapping['amount_rur']
        transactions['client_day_of_week_daily_group_average'] = baseline.map(mapping)

    # processing day of month

    # transaction averages

    transactions['day_of_month'] = transactions['trans_date'].dt.day

    baseline = transactions['day_of_month']
    mapping = transactions.groupby('day_of_month').mean()['amount_rur']
    transactions['general_day_of_month_transaction_average'] = baseline.map(mapping)

    baseline = pd.Series(list(zip(transactions['day_of_month'], transactions['small_group'])))
    mapping = transactions.groupby(['day_of_month', 'small_group']).mean()['amount_rur']
    transactions['general_day_of_month_transaction_group_average'] = baseline.map(mapping)

    if leak_client_id:
        baseline = pd.Series(list(zip(transactions['client_id'], transactions['day_of_month'])))
        mapping = transactions.groupby(['client_id','day_of_month']).mean()['amount_rur']
        transactions['client_day_of_month_transaction_average'] = baseline.map(mapping)

        baseline = pd.Series(list(zip(transactions['client_id'], transactions['day_of_month'], transactions['small_group'])))
        mapping = transactions.groupby(['client_id','day_of_month', 'small_group']).mean()['amount_rur']
        transactions['client_day_of_month_transaction_group_average'] = baseline.map(mapping)

    # daily averages

    day_of_month_counts = pd.Series(transactions['trans_date'].unique()).dt.day.value_counts()
    unique_clients = transactions['client_id'].nunique()

    baseline = transactions['day_of_month']
    mapping = transactions.groupby('day_of_month').sum()['amount_rur']
    mapping /= (day_of_month_counts * unique_clients)
    transactions['general_day_of_month_daily_average'] = baseline.map(mapping)

    baseline = pd.Series(list(zip(transactions['day_of_month'], transactions['small_group'])))
    mapping = transactions.groupby(['day_of_month', 'small_group']).sum()['amount_rur']
    mapping = mapping.reset_index()
    mapping['amount_rur'] /= (mapping['day_of_month'].map(day_of_month_counts) * unique_clients)
    mapping = mapping.set_index(['day_of_month', 'small_group'])
    mapping = mapping['amount_rur']
    transactions['general_day_of_month_daily_group_average'] = baseline.map(mapping)
    
    if leak_client_id:
        baseline = pd.Series(list(zip(transactions['client_id'], transactions['day_of_month'])))
        mapping = transactions.groupby(['client_id', 'day_of_month']).sum()['amount_rur']
        mapping = mapping.reset_index()
        mapping['amount_rur'] /= (mapping['day_of_month'].map(day_of_month_counts))
        mapping = mapping.set_index(['client_id', 'day_of_month'])
        mapping = mapping['amount_rur']
        transactions['client_day_of_month_daily_average'] = baseline.map(mapping)

        baseline = pd.Series(list(zip(transactions['client_id'], transactions['day_of_month'], transactions['small_group'])))
        mapping = transactions.groupby(['client_id', 'day_of_month', 'small_group']).sum()['amount_rur']
        mapping = mapping.reset_index()
        mapping['amount_rur'] /= (mapping['day_of_month'].map(day_of_month_counts))
        mapping = mapping.set_index(['client_id', 'day_of_month', 'small_group'])
        mapping = mapping['amount_rur']
        transactions['client_day_of_month_daily_group_average'] = baseline.map(mapping)

    # processing month

    # transaction averages

    transactions['month'] = transactions['trans_date'].dt.month

    baseline = transactions['month']
    mapping = transactions.groupby('month').mean()['amount_rur']
    transactions['general_month_transaction_average'] = baseline.map(mapping)

    baseline = pd.Series(list(zip(transactions['month'], transactions['small_group'])))
    mapping = transactions.groupby(['month', 'small_group']).mean()['amount_rur']
    transactions['general_month_transaction_group_average'] = baseline.map(mapping)

    if leak_client_id:
        baseline = pd.Series(list(zip(transactions['client_id'], transactions['month'])))
        mapping = transactions.groupby(['client_id','month']).mean()['amount_rur']
        transactions['client_month_transaction_average'] = baseline.map(mapping)

        baseline = pd.Series(list(zip(transactions['client_id'], transactions['month'], transactions['small_group'])))
        mapping = transactions.groupby(['client_id','month', 'small_group']).mean()['amount_rur']
        transactions['client_month_transaction_group_average'] = baseline.map(mapping)
    

    # daily averages

    month_counts = pd.Series(transactions['trans_date'].unique()).dt.month.value_counts()
    unique_clients = transactions['client_id'].nunique()

    baseline = transactions['month']
    mapping = transactions.groupby('month').sum()['amount_rur']
    mapping /= (month_counts * unique_clients)
    transactions['general_month_daily_average'] = baseline.map(mapping)

    baseline = pd.Series(list(zip(transactions['month'], transactions['small_group'])))
    mapping = transactions.groupby(['month', 'small_group']).sum()['amount_rur']
    mapping = mapping.reset_index()
    mapping['amount_rur'] /= (mapping['month'].map(month_counts) * unique_clients)
    mapping = mapping.set_index(['month', 'small_group'])
    mapping = mapping['amount_rur']
    transactions['general_month_daily_group_average'] = baseline.map(mapping)

    if leak_client_id:
        baseline = pd.Series(list(zip(transactions['client_id'], transactions['month'])))
        mapping = transactions.groupby(['client_id', 'month']).sum()['amount_rur']
        mapping = mapping.reset_index()
        mapping['amount_rur'] /= (mapping['month'].map(month_counts))
        mapping = mapping.set_index(['client_id', 'month'])
        mapping = mapping['amount_rur']
        transactions['client_month_daily_average'] = baseline.map(mapping)

        baseline = pd.Series(list(zip(transactions['client_id'], transactions['month'], transactions['small_group'])))
        mapping = transactions.groupby(['client_id', 'month', 'small_group']).sum()['amount_rur']
        mapping = mapping.reset_index()
        mapping['amount_rur'] /= (mapping['month'].map(month_counts))
        mapping = mapping.set_index(['client_id', 'month', 'small_group'])
        mapping = mapping['amount_rur']
        transactions['client_month_daily_group_average'] = baseline.map(mapping)

In [9]:
def interval_analysis(transactions, time_intervals):

    # Analyses past transactions in given intervals
    # Computes slowly and leaks client_id, since it analyses transactions of every client
    # time_intervals is a list of strings, each string is a number of days with the letter "d" at the end
    # example: ['1d', '2d', '7d', '14d', '30d', '365d']

    for time_interval in tqdm(time_intervals, desc='Time intervals processed:'):
        transactions['past_' + time_interval + '_interval_max'] = 0
        transactions['past_' + time_interval + '_interval_avg'] = 0
        transactions['past_' + time_interval + '_interval_sum'] = 0
        transactions['past_' + time_interval + '_interval_cnt'] = 0
        transactions['past_' + time_interval + '_interval_group_max'] = 0
        transactions['past_' + time_interval + '_interval_group_avg'] = 0
        transactions['past_' + time_interval + '_interval_group_sum'] = 0
        transactions['past_' + time_interval + '_interval_group_cnt'] = 0
        for client in tqdm(transactions['client_id'].unique(), desc='Clients processed:'):
            client_slice_idx = transactions[transactions['client_id'] == client].index
            client_slice = transactions.loc[client_slice_idx,:]
            transactions.loc[client_slice_idx, 'past_' + time_interval + '_interval_max'] = client_slice.set_index('trans_date')['amount_rur'].rolling(time_interval).max().reset_index()['amount_rur']
            transactions.loc[client_slice_idx, 'past_' + time_interval + '_interval_avg'] = client_slice.set_index('trans_date')['amount_rur'].rolling(time_interval).mean().reset_index()['amount_rur']
            transactions.loc[client_slice_idx, 'past_' + time_interval + '_interval_sum'] = client_slice.set_index('trans_date')['amount_rur'].rolling(time_interval).sum().reset_index()['amount_rur']
            transactions.loc[client_slice_idx, 'past_' + time_interval + '_interval_cnt'] = client_slice.set_index('trans_date')['amount_rur'].rolling(time_interval).count().reset_index()['amount_rur']

            for group in client_slice['small_group'].unique():
                group_slice_idx = transactions[((transactions['client_id'] == client) & (transactions['small_group'] == group))].index
                group_slice = transactions.loc[group_slice_idx,:]
                transactions.loc[group_slice_idx, 'past_' + time_interval + '_interval_group_max'] = np.array(group_slice.set_index('trans_date')['amount_rur'].rolling(time_interval).max().reset_index()['amount_rur'])
                transactions.loc[group_slice_idx, 'past_' + time_interval + '_interval_group_avg'] = np.array(group_slice.set_index('trans_date')['amount_rur'].rolling(time_interval).mean().reset_index()['amount_rur'])
                transactions.loc[group_slice_idx, 'past_' + time_interval + '_interval_group_sum'] = np.array(group_slice.set_index('trans_date')['amount_rur'].rolling(time_interval).sum().reset_index()['amount_rur'])
                transactions.loc[group_slice_idx, 'past_' + time_interval + '_interval_group_cnt'] = np.array(group_slice.set_index('trans_date')['amount_rur'].rolling(time_interval).count().reset_index()['amount_rur'])

In [40]:
def process(data, inplace = False, leak_client_id=False):

    # Processes the dataset, downsampling, filtering some of the feature values
    # and adding new features, based on the EDA conducted earlier.
    # If leak_client_id is set to True, features based on client_id will be added to the data.
    # This might be unwanted if client_id is the target (like it is in a metric learning task, in a way).

    if not inplace:
        transactions = data.copy(deep=True)
    else:
        transactions = data.copy(deep=False)

    downsample(transactions, verbose=False)
    filter_groups(transactions, threshold_value=0.01, based_on_volume=True)
    flag_large_transactions(transactions, quantile=0.996)
    group_mean(transactions)
    process_datetime(transactions, leak_client_id=False)
    
    if leak_client_id:
        interval_analysis(transactions, time_intervals)
        
    post_downsample(transactions)

    return transactions  


In [39]:
def post_downsample(transactions, verbose=False):
    if verbose:
        print('before downsampling:\n', transactions.dtypes)

    for column in transactions.columns:
        old_type = transactions[column].dtype
        if ((transactions[column].dtype == 'int16') or 
            (transactions[column].dtype == 'int32') or
            (transactions[column].dtype == 'int64')):
                transactions[column] = pd.to_numeric(transactions[column], downcast='signed')
        if ((transactions[column].dtype == 'float16') or 
            (transactions[column].dtype == 'float32') or
            (transactions[column].dtype == 'float64')):
                transactions[column] = pd.to_numeric(transactions[column], downcast='float')
        if (verbose) and (old_type != transactions[column].dtype):
            print("downsampled", column, "from", old_type, "to", transactions[column].dtype)
    if verbose:
        print('after downsampling:\n', transactions.dtypes)

In [11]:
# %%timeit
# This will take a while. 
# We have a lot of data, after all.
# Get yourself a nice cup of tea while waiting.

transactions_train_processed = process(transactions_train)
transactions_test_processed = process(transactions_test)

In [43]:
transactions_train_processed.to_csv(path+'transactions_train_processed.csv', index=False)
transactions_test_processed.to_csv(path+'transactions_test_processed.csv', index=False)